# Black-Swan Hunter Trading Bot Demo

This notebook demonstrates the complete Black Swan Hunter system:
1. Feature generation for dual models
2. MFE labeling and tail event classification
3. Model training (XGB + LSTM)
4. Backtesting with comprehensive metrics
5. Performance analysis and visualization

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
ROOT = Path().resolve().parent
sys.path.append(str(ROOT))

# Import Black Swan components
from src.data.database import TradingDatabase
from src.features.black_swan_pipeline import BlackSwanFeaturePipeline
from src.features.black_swan_labeling import BlackSwanLabeling
from src.models.xgb_mfe_model import XGBMFERegressor
from src.models.lstm_tail_model import LSTMTailClassifier
from src.backtesting.black_swan_backtest import BlackSwanBacktester, BacktestConfig

print("Black Swan Hunter components loaded successfully!")

## 1. Load Market Data

In [ ]:
# Load EURUSD M5 data
symbol = 'EURUSDm'
db_path = ROOT / 'data' / 'trading_system.db'
db = TradingDatabase(str(db_path))

# Load recent data
with db.get_connection() as conn:
    query = """
    SELECT time, open, high, low, close, IFNULL(volume, 0) as volume
    FROM bars 
    WHERE symbol = ?
    ORDER BY time DESC
    LIMIT 50000
    """
    df = pd.read_sql_query(query, conn, params=[symbol], parse_dates=['time'])
    df = df.sort_values('time').set_index('time')

print(f"Loaded {len(df)} bars for {symbol}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
df.head()

## 2. Generate Features and Labels

In [ ]:
# Initialize pipelines
feature_pipeline = BlackSwanFeaturePipeline()
labeling_pipeline = BlackSwanLabeling(forecast_horizon=100)

# Generate XGB features
print("Generating XGB features...")
xgb_features = feature_pipeline.generate_xgb_features(df, symbol)
print(f"XGB features shape: {xgb_features.shape}")

# Generate LSTM features
print("Generating LSTM features...")
lstm_features = feature_pipeline.generate_lstm_features(df, symbol)
print(f"LSTM features shape: {lstm_features.shape}")

# Generate labels
print("Generating MFE labels and tail classifications...")
labels_dict = labeling_pipeline.generate_labels_for_symbol(df, symbol)

for label_type, labels_df in labels_dict.items():
    print(f"{label_type}: {labels_df.shape}")

## 3. Analyze Label Distributions

In [ ]:
# Analyze MFE distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# MFE distribution for long trades
mfe_long = labels_dict['xgb_long']['mfe_target']
axes[0,0].hist(mfe_long, bins=50, alpha=0.7, color='green')
axes[0,0].set_title('MFE Distribution - Long Trades')
axes[0,0].set_xlabel('MFE (R-multiples)')
axes[0,0].axvline(mfe_long.mean(), color='red', linestyle='--', label=f'Mean: {mfe_long.mean():.2f}R')
axes[0,0].legend()

# MFE distribution for short trades
mfe_short = labels_dict['xgb_short']['mfe_target']
axes[0,1].hist(mfe_short, bins=50, alpha=0.7, color='red')
axes[0,1].set_title('MFE Distribution - Short Trades')
axes[0,1].set_xlabel('MFE (R-multiples)')
axes[0,1].axvline(mfe_short.mean(), color='green', linestyle='--', label=f'Mean: {mfe_short.mean():.2f}R')
axes[0,1].legend()

# Tail class distribution for long
tail_long = labels_dict['lstm_long']['tail_class']
tail_counts_long = tail_long.value_counts().sort_index()
axes[1,0].bar(tail_counts_long.index, tail_counts_long.values, alpha=0.7, color='green')
axes[1,0].set_title('Tail Class Distribution - Long')
axes[1,0].set_xlabel('Tail Class')
axes[1,0].set_ylabel('Count')
for i, v in enumerate(tail_counts_long.values):
    axes[1,0].text(i, v, f'{v}\n({v/len(tail_long)*100:.1f}%)', ha='center', va='bottom')

# Tail class distribution for short
tail_short = labels_dict['lstm_short']['tail_class']
tail_counts_short = tail_short.value_counts().sort_index()
axes[1,1].bar(tail_counts_short.index, tail_counts_short.values, alpha=0.7, color='red')
axes[1,1].set_title('Tail Class Distribution - Short')
axes[1,1].set_xlabel('Tail Class')
axes[1,1].set_ylabel('Count')
for i, v in enumerate(tail_counts_short.values):
    axes[1,1].text(i, v, f'{v}\n({v/len(tail_short)*100:.1f}%)', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Print statistics
print("\n=== MFE Statistics ===")
print(f"Long trades - Mean: {mfe_long.mean():.2f}R, Std: {mfe_long.std():.2f}R, Max: {mfe_long.max():.2f}R")
print(f"Short trades - Mean: {mfe_short.mean():.2f}R, Std: {mfe_short.std():.2f}R, Max: {mfe_short.max():.2f}R")

print("\n=== Tail Event Statistics ===")
extreme_long = (tail_long >= 3).sum()
extreme_short = (tail_short >= 3).sum()
print(f"Extreme tail events (Class 3): Long={extreme_long} ({extreme_long/len(tail_long)*100:.3f}%), Short={extreme_short} ({extreme_short/len(tail_short)*100:.3f}%)")

## 4. Train Models (Simplified Demo)

In [ ]:
# Prepare training data
from sklearn.model_selection import train_test_split

# XGB training data (combine long and short)
xgb_labels_combined = pd.concat([labels_dict['xgb_long'], labels_dict['xgb_short']]).sort_index()
common_idx_xgb = xgb_features.index.intersection(xgb_labels_combined.index)
X_xgb = xgb_features.loc[common_idx_xgb]
y_xgb = xgb_labels_combined.loc[common_idx_xgb]['mfe_target']

# Remove symbol column
X_xgb_numeric = X_xgb.drop('symbol', axis=1)

# Split for demo (in production, use time-based splits)
X_xgb_train, X_xgb_test, y_xgb_train, y_xgb_test = train_test_split(
    X_xgb_numeric, y_xgb, test_size=0.2, random_state=42
)

print(f"XGB training data: {X_xgb_train.shape}, Test: {X_xgb_test.shape}")
print(f"XGB target stats - Train mean: {y_xgb_train.mean():.2f}R, Test mean: {y_xgb_test.mean():.2f}R")

In [ ]:
# Train XGBoost MFE model (simplified)
print("Training XGBoost MFE model...")
xgb_model = XGBMFERegressor()

# Simple training without full CV for demo
from sklearn.model_selection import KFold
simple_cv = KFold(n_splits=3, shuffle=False)

# Note: In production, use the full training pipeline with purged CV
xgb_model.model = xgb_model.__class__.__bases__[0](**xgb_model.params)
X_xgb_processed = xgb_model.prepare_features(X_xgb_train, fit_scaler=True)
xgb_model.model.fit(X_xgb_processed, y_xgb_train.values)
xgb_model.is_fitted = True

# Evaluate
xgb_eval = xgb_model.evaluate(X_xgb_test, y_xgb_test.values)
print(f"XGB Test RMSE: {xgb_eval['rmse']:.3f}, R²: {xgb_eval['r2_score']:.3f}")

# Feature importance
importance = xgb_model.get_feature_importance()
top_features = importance['top_10_features']
print("\nTop 5 XGB features:")
for i, (feature, score) in enumerate(top_features[:5]):
    print(f"{i+1}. {feature}: {score:.4f}")

## 5. Simulate Predictions for Backtesting

In [ ]:
# Generate predictions for backtesting
print("Generating predictions for backtest...")

# Use test set for predictions
xgb_predictions = xgb_model.predict(X_xgb_test)

# Create predictions DataFrame
predictions_df = pd.DataFrame({
    'mfe_prediction': xgb_predictions,
    'tail_prob_0': 0.7,  # Simplified - normally from LSTM
    'tail_prob_1': 0.2,
    'tail_prob_2': 0.08,
    'tail_prob_3': 0.02
}, index=X_xgb_test.index)

print(f"Generated {len(predictions_df)} predictions")
print(f"MFE prediction stats: Mean={predictions_df['mfe_prediction'].mean():.2f}R, "
      f"Std={predictions_df['mfe_prediction'].std():.2f}R, "
      f"Max={predictions_df['mfe_prediction'].max():.2f}R")

# Plot prediction distribution
plt.figure(figsize=(10, 6))
plt.hist(predictions_df['mfe_prediction'], bins=50, alpha=0.7, color='blue')
plt.axvline(5.0, color='red', linestyle='--', label='Min Entry Threshold (5R)')
plt.axvline(predictions_df['mfe_prediction'].mean(), color='green', linestyle='--', 
           label=f'Mean: {predictions_df["mfe_prediction"].mean():.2f}R')
plt.title('XGB MFE Predictions Distribution')
plt.xlabel('Predicted MFE (R-multiples)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## 6. Run Backtest

In [ ]:
# Configure backtest
config = BacktestConfig(
    initial_capital=100000.0,
    base_risk_per_trade=0.005,  # 0.5% risk per trade
    max_concurrent_positions=3,
    min_mfe_prediction=5.0,
    min_tail_probability=0.3,
    stop_loss_atr_multiple=1.0,
    partial_take_profit_r=5.0,
    max_hold_bars=500
)

print("Running backtest...")
backtester = BlackSwanBacktester(config)

# Prepare market data for backtest
backtest_data = df.loc[predictions_df.index]

# Add required technical indicators
from src.features.technical_indicators import TechnicalIndicators
ti = TechnicalIndicators()
backtest_data['atr14'] = ti.calculate_atr(backtest_data['high'], backtest_data['low'], backtest_data['close'], 14)
backtest_data['ema20'] = ti.calculate_ema(backtest_data['close'], 20)
backtest_data['ema50'] = ti.calculate_ema(backtest_data['close'], 50)
backtest_data['ema200'] = ti.calculate_ema(backtest_data['close'], 200)

# Run backtest
results = backtester.run_backtest(backtest_data, predictions_df)

print("\n=== Backtest Results ===")
metrics = results['performance_metrics']
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

## 7. Analyze Backtest Performance

In [ ]:
# Plot equity curve
equity_df = pd.DataFrame(results['equity_curve'])
equity_df['timestamp'] = pd.to_datetime(equity_df['timestamp'])
equity_df.set_index('timestamp', inplace=True)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Equity curve
axes[0,0].plot(equity_df.index, equity_df['equity'])
axes[0,0].axhline(config.initial_capital, color='red', linestyle='--', alpha=0.7, label='Initial Capital')
axes[0,0].set_title('Equity Curve')
axes[0,0].set_ylabel('Account Value ($)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Drawdown
equity_values = equity_df['equity'].values
peak = np.maximum.accumulate(equity_values)
drawdown = (peak - equity_values) / peak * 100
axes[0,1].fill_between(equity_df.index, 0, -drawdown, alpha=0.7, color='red')
axes[0,1].set_title('Drawdown (%)')
axes[0,1].set_ylabel('Drawdown (%)')
axes[0,1].grid(True, alpha=0.3)

# Trade P&L distribution
trades_df = pd.DataFrame(results['trades'])
if not trades_df.empty:
    pnl_r = trades_df['pnl_r_multiple'].dropna()
    axes[1,0].hist(pnl_r, bins=30, alpha=0.7, color='blue')
    axes[1,0].axvline(0, color='red', linestyle='--', alpha=0.7)
    axes[1,0].axvline(pnl_r.mean(), color='green', linestyle='--', alpha=0.7, 
                     label=f'Mean: {pnl_r.mean():.2f}R')
    axes[1,0].set_title('Trade P&L Distribution')
    axes[1,0].set_xlabel('P&L (R-multiples)')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Cumulative P&L
    trades_df['entry_time'] = pd.to_datetime(trades_df['entry_time'])
    trades_df = trades_df.sort_values('entry_time')
    trades_df['cumulative_pnl_r'] = trades_df['pnl_r_multiple'].cumsum()
    
    axes[1,1].plot(trades_df['entry_time'], trades_df['cumulative_pnl_r'])
    axes[1,1].axhline(0, color='red', linestyle='--', alpha=0.7)
    axes[1,1].set_title('Cumulative P&L (R-multiples)')
    axes[1,1].set_ylabel('Cumulative P&L (R)')
    axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Trade analysis
if not trades_df.empty:
    print("\n=== Trade Analysis ===")
    winning_trades = trades_df[trades_df['pnl_r_multiple'] > 0]
    losing_trades = trades_df[trades_df['pnl_r_multiple'] <= 0]
    
    print(f"Total trades: {len(trades_df)}")
    print(f"Winning trades: {len(winning_trades)} ({len(winning_trades)/len(trades_df)*100:.1f}%)")
    print(f"Losing trades: {len(losing_trades)} ({len(losing_trades)/len(trades_df)*100:.1f}%)")
    
    if len(winning_trades) > 0:
        print(f"Average winning trade: {winning_trades['pnl_r_multiple'].mean():.2f}R")
        print(f"Largest winning trade: {winning_trades['pnl_r_multiple'].max():.2f}R")
    
    if len(losing_trades) > 0:
        print(f"Average losing trade: {losing_trades['pnl_r_multiple'].mean():.2f}R")
        print(f"Largest losing trade: {losing_trades['pnl_r_multiple'].min():.2f}R")
    
    # Tail events captured
    tail_events = trades_df[trades_df['pnl_r_multiple'] >= 5.0]
    extreme_tail_events = trades_df[trades_df['pnl_r_multiple'] >= 20.0]
    
    print(f"\nTail events captured (≥5R): {len(tail_events)}")
    print(f"Extreme tail events captured (≥20R): {len(extreme_tail_events)}")
    
    if len(tail_events) > 0:
        print(f"Average tail event: {tail_events['pnl_r_multiple'].mean():.2f}R")
        print(f"Largest tail event: {tail_events['pnl_r_multiple'].max():.2f}R")

## 8. Model Performance Analysis

In [ ]:
# Analyze prediction accuracy
if not trades_df.empty:
    # Compare predicted vs realized MFE
    trades_with_pred = trades_df.dropna(subset=['mfe_prediction', 'max_favorable_excursion'])
    
    if len(trades_with_pred) > 0:
        # Convert MFE to R-multiples
        trades_with_pred['realized_mfe_r'] = trades_with_pred['max_favorable_excursion'] / trades_with_pred['atr_at_entry']
        
        plt.figure(figsize=(12, 5))
        
        # Predicted vs Realized MFE scatter plot
        plt.subplot(1, 2, 1)
        plt.scatter(trades_with_pred['mfe_prediction'], trades_with_pred['realized_mfe_r'], alpha=0.6)
        plt.plot([0, 50], [0, 50], 'r--', alpha=0.7, label='Perfect Prediction')
        plt.xlabel('Predicted MFE (R)')
        plt.ylabel('Realized MFE (R)')
        plt.title('Predicted vs Realized MFE')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Prediction error distribution
        plt.subplot(1, 2, 2)
        prediction_error = trades_with_pred['realized_mfe_r'] - trades_with_pred['mfe_prediction']
        plt.hist(prediction_error, bins=20, alpha=0.7, color='orange')
        plt.axvline(0, color='red', linestyle='--', alpha=0.7)
        plt.axvline(prediction_error.mean(), color='green', linestyle='--', alpha=0.7,
                   label=f'Mean Error: {prediction_error.mean():.2f}R')
        plt.xlabel('Prediction Error (Realized - Predicted)')
        plt.ylabel('Frequency')
        plt.title('MFE Prediction Error Distribution')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Calculate prediction metrics
        from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
        
        rmse = np.sqrt(mean_squared_error(trades_with_pred['realized_mfe_r'], trades_with_pred['mfe_prediction']))
        r2 = r2_score(trades_with_pred['realized_mfe_r'], trades_with_pred['mfe_prediction'])
        mae = mean_absolute_error(trades_with_pred['realized_mfe_r'], trades_with_pred['mfe_prediction'])
        
        print("\n=== Prediction Performance ===")
        print(f"RMSE: {rmse:.3f}R")
        print(f"R²: {r2:.3f}")
        print(f"MAE: {mae:.3f}R")
        print(f"Mean prediction error: {prediction_error.mean():.3f}R")
        print(f"Std prediction error: {prediction_error.std():.3f}R")
        
        # Directional accuracy
        correct_direction = ((trades_with_pred['mfe_prediction'] >= 5.0) & 
                           (trades_with_pred['realized_mfe_r'] >= 5.0)).sum()
        total_predictions = len(trades_with_pred)
        directional_accuracy = correct_direction / total_predictions * 100
        
        print(f"\nDirectional accuracy (≥5R): {directional_accuracy:.1f}% ({correct_direction}/{total_predictions})")

## Summary

This demo showcased the Black Swan Hunter trading system:

1. **Feature Engineering**: Generated 25+ XGB features and 10+ LSTM features with ATR normalization
2. **Labeling**: Created MFE regression targets and tail event classifications
3. **Model Training**: Trained XGBoost for MFE prediction (LSTM training requires more data)
4. **Backtesting**: Comprehensive walk-forward simulation with risk management
5. **Analysis**: Performance metrics, trade analysis, and prediction accuracy

### Key Insights:
- The system focuses on detecting extreme price movements (tail events)
- Risk management is ATR-based for consistent R-multiple analysis
- Position sizing adapts to prediction confidence
- Comprehensive performance tracking enables continuous improvement

### Next Steps:
1. Train LSTM model with sufficient data and proper validation
2. Implement full walk-forward backtesting with expanding windows
3. Optimize hyperparameters using Optuna
4. Deploy to live trading with MT5 integration
5. Monitor model performance and retrain regularly